In [34]:
# ! Taken from https://github.com/locuslab/DC3/blob/main/datasets/simple/make_dataset.py
# ! To test whether there is 'unconstrained' optimization due to quadratic minimum lying in the feasible region.

import numpy as np
import pickle
import torch

import sys
import os
sys.path.insert(1, os.path.join(sys.path[0], os.pardir, os.pardir))
from utils import SimpleProblem

import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

torch.set_default_dtype(torch.float64)

num_var = 100
num_ineq = 50
num_eq = 50
num_examples = 10

print(num_ineq, num_eq)
np.random.seed(17)
Q = np.diag(np.random.random(num_var))
p = np.random.random(num_var)
A = np.random.normal(loc=0, scale=1., size=(num_eq, num_var))
X = np.random.uniform(-1, 1, size=(num_examples, num_eq))
G = np.random.normal(loc=0, scale=1., size=(num_ineq, num_var))
h = -np.sum(np.abs(G@np.linalg.pinv(A)), axis=1)
print(G)
problem = SimpleProblem(Q, p, A, G, h, X)
problem.calc_Y()
print(len(problem.Y))

# with open("./random_simple_dataset_var{}_ineq{}_eq{}_ex{}".format(num_var, num_ineq, num_eq, num_examples), 'wb') as f:
#     pickle.dump(problem, f)

50 50
[[ 0.17256248  1.63814966  1.61112181 ...  0.25319614 -1.03376536
   2.03053609]
 [-0.12021127 -0.50695122  1.22999721 ...  0.34333568  1.40611853
  -0.81996443]
 [ 0.36198438  0.10745732  0.07060932 ...  0.6487779   0.58438818
   0.03035542]
 ...
 [ 1.14805999 -1.06828696 -1.08601048 ...  1.39941345  0.55500201
  -0.97364319]
 [-0.05520414 -0.93310946 -0.77991719 ... -1.71230435  0.98146462
   1.23164395]
 [ 0.63486058  1.70115442  1.07583429 ... -1.64248858 -0.55329358
  -1.40723231]]
running osqp
10


In [30]:
# ! Test whether the quadratic minimum is inside of the feasible region

# Quadratic minimum
y_min = -np.linalg.inv(Q) @ p
print(y_min)
# Inequality residual
ineq_resid = problem.ineq_resid(torch.tensor(X), torch.tensor(y_min))

print(ineq_resid)

print((ineq_resid > 0).sum())
# Check that not all constraints should be satisfied.
assert (ineq_resid > 0).sum() > 0

[-2.52822427e+00 -7.78501581e-01 -2.68746459e+00 -1.24674084e+01
 -1.09133154e+00 -4.73088154e-01 -1.87266637e-01 -9.05067516e-01
 -1.14929993e+01 -2.50769469e+00 -5.21424453e-01 -8.40170212e+00
 -4.22576224e-01 -8.92527729e-01 -1.85164123e+01 -1.99281581e-01
 -1.19820758e+00 -2.74022890e-01 -6.38925492e-01 -3.41579715e+00
 -1.62181043e+00 -9.63025591e-01 -1.79567665e+00 -8.71600261e-01
 -8.83405662e-02 -6.17196919e+00 -1.24967424e+00 -1.41565649e+01
 -8.96786584e-01 -1.66052059e+00 -4.67387859e+00 -3.24624879e-01
 -1.43343612e+00 -8.44114878e+00 -4.32168455e-01 -5.99717049e+00
 -8.65053556e-01 -4.92154404e+00 -3.53282994e-01 -4.78883869e-01
 -2.94784180e-01 -8.83375347e+00 -3.75325789e-01 -2.56177286e-01
 -4.83107962e-01 -2.88493441e-01 -8.95004797e-01 -5.91067507e+00
 -2.11770579e-01 -8.95431562e-01 -6.74017819e-02 -7.97305804e+00
 -1.66531483e+00 -1.64262662e+00 -9.22045973e-01 -6.84993682e-01
 -2.98787422e+00 -8.72421335e-01 -7.78337198e+00 -1.50995004e+00
 -1.62961360e+00 -9.80505

In [14]:
# ! Taken from https://github.com/locuslab/DC3/blob/main/datasets/simple/make_dataset.py

import numpy as np
import pickle
import torch

import sys
import os
sys.path.insert(1, os.path.join(sys.path[0], os.pardir, os.pardir))
from utils import SimpleProblem

import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

torch.set_default_dtype(torch.float64)

num_var = 100
num_ineq = 50
num_eq = 50
num_examples = 100

print(num_ineq, num_eq)
np.random.seed(17)
Q = np.diag(np.random.random(num_var)) 
p = np.random.random(num_var) 
A = np.random.normal(loc=0, scale=1., size=(num_eq, num_var))
X = np.random.uniform(-1, 0, size=(num_examples, num_eq))
# H = np.random.uniform(-1, 1, size=(num_examples, num_eq))

# Generate a base G matrix
G_base = np.random.normal(loc=0, scale=1., size=(num_ineq, num_var))

# Generate a G matrix for each example with only the first 10 rows different, to mimic GEP
G_list = []
for _ in range(num_examples):
    G_sample = G_base.copy()
    G_sample[:10, :] = np.random.normal(loc=0, scale=1., size=(10, num_var))  # First 10 rows unique
    G_list.append(G_sample)

# Create H matrix for each example
H_list = []
for G in G_list:
    # H = np.abs(G @ np.linalg.pinv(A)) 
    # H = np.sum(H @ X.T, axis=1)
    # H = np.sum(H, axis=1)
    H = np.random.uniform(-1, 1, size=(num_eq))
    print(H)
    H_list.append(H)

# Convert H_list to a numpy array with correct shape
H = np.stack(H_list, axis=0)  # Shape (num_examples, num_ineq)
# Check dimensions
G = np.array(G_list)
print(f"Shape of X: {X.shape}")
print(f"Shape of H: {H.shape}")
print(f"Shape of G: {G.shape}")
print(f"Shape of A: {A.shape}")
# print(f"Shape of each H: {H_list[0].shape}")

problem = SimpleProblem(Q, p, A, G, H, X, modified_problem=True)
problem.calc_Y()
print(len(problem.Y))

with open("./QP_data/CUSTOM_random_simple_dataset_var{}_ineq{}_eq{}_ex{}".format(num_var, num_ineq, num_eq, num_examples), 'wb') as f:
    pickle.dump(problem, f)

50 50
[ 0.15895363  0.54900018  0.40562359 -0.247587   -0.90456476 -0.13154054
 -0.23358917  0.32678092  0.26584206 -0.73602043 -0.93147105 -0.27517845
  0.9663737   0.27515106  0.80722495  0.87484996 -0.00625296 -0.27443801
  0.20005481 -0.36583222 -0.28960495 -0.32653826  0.28419638  0.9412314
  0.99284311 -0.15148096  0.45057723  0.35341294 -0.82760804  0.06983223
 -0.19270828 -0.18933486  0.75132882 -0.19930246 -0.48770609 -0.09969159
 -0.09179319  0.49589908 -0.36990724 -0.03026224 -0.25482762  0.703467
 -0.95690789  0.31268868 -0.92021322 -0.88286705 -0.84425939  0.35509431
 -0.53530414 -0.97562577]
[-0.18951334 -0.82543014  0.58929164  0.00588357 -0.05518619  0.94491197
 -0.94135362 -0.71722674  0.82240527 -0.77707445  0.50852518 -0.00665039
 -0.13816708 -0.83822157 -0.89884661  0.63821256 -0.20245336  0.98110194
 -0.24133853  0.13186683 -0.07744686 -0.38928071  0.60740829 -0.02516465
  0.31836283  0.59548645  0.44062361 -0.59887726 -0.05086563 -0.47601684
  0.38423094  0.951148